In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

d:\projects\sentiment_analysis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("sismetanin/rubert-ru-sentiment-rusentiment")
model = AutoModelForSequenceClassification.from_pretrained("sismetanin/rubert-ru-sentiment-rusentiment")




In [25]:
inputs = tokenizer("Очень плохие наушники", padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    print(torch.nn.functional.softmax(logits))

tensor([[0.1847, 0.7454, 0.0160, 0.0526, 0.0013]])


C:\Users\пк\AppData\Local\Temp\ipykernel_15136\4150916011.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(torch.nn.functional.softmax(logits))


In [26]:
texts = [
    "Этот фильм был просто ужасен!",  # ожидаем негатив
    "Фильм нормальный, не хуже других.",  # ожидаем нейтральный
    "Мне очень понравилось! Это шедевр!",  # ожидаем очень позитивный
]

for text in texts:
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    
    print(f"Текст: {text}")
    print(f"Логиты: {logits}")
    print(f"Предсказанный класс: {predicted_class} ({['Neutral', 'Positive', 'Very Positive', 'Negative', 'Very Negative'][predicted_class]})")
    print("-" * 50)

Текст: Этот фильм был просто ужасен!
Логиты: tensor([[ 2.8753,  0.2771,  0.2848,  0.5628, -3.2889]])
Предсказанный класс: 0 (Neutral)
--------------------------------------------------
Текст: Фильм нормальный, не хуже других.
Логиты: tensor([[-0.5001,  0.2734,  3.5308, -0.1880, -3.1131]])
Предсказанный класс: 2 (Very Positive)
--------------------------------------------------
Текст: Мне очень понравилось! Это шедевр!
Логиты: tensor([[-0.9837, -1.5925,  4.6931, -1.1424, -1.2629]])
Предсказанный класс: 2 (Very Positive)
--------------------------------------------------


In [2]:
import pandas as pd

df = pd.read_csv("reviews.csv", sep="\t")
print(df.head())


                                              review sentiment
0  качество плохое пошив ужасный (горловина напер...  negative
1  Товар отдали другому человеку, я не получила п...  negative
2  Ужасная синтетика! Тонкая, ничего общего с пре...  negative
3  товар не пришел, продавец продлил защиту без м...  negative
4      Кофточка голая синтетика, носить не возможно.  negative


In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import DataCollatorWithPadding

# Класс датасета
# Класс датасета
def load_dataset(csv_path):
    df = pd.read_csv(csv_path, sep="\t")
    label_mapping = {"negative": 0, "neautral": 1, "positive": 2}
    df["sentiment"] = df["sentiment"].map(label_mapping)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Перемешивание
    df = df.head(1000) 
    train_texts, test_texts, train_labels, test_labels = train_test_split(
        df['review'], df['sentiment'], test_size=0.2, random_state=42
    )
    return SentimentDataset(train_texts.tolist(), train_labels.tolist()), SentimentDataset(test_texts.tolist(), test_labels.tolist())


class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        return {"input_ids": encoding["input_ids"].squeeze(0),
                "attention_mask": encoding["attention_mask"].squeeze(0),
                "label": torch.tensor(self.labels[idx], dtype=torch.long)}

# Выбор модели
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 класса (negative, neutral, positive)

# Подготовка датасета
train_dataset, test_dataset = load_dataset("reviews.csv")
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Определение метрик
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {"accuracy": acc, "f1": f1}

# Параметры обучения
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,  # Увеличено для ускорения
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
)



data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,  # Замена устаревшего tokenizer
    compute_metrics=compute_metrics,
)

trainer.train()

# Сохранение модели
model.save_pretrained("./finetuned_rubert")
tokenizer.save_pretrained("./finetuned_rubert")


d:\projects\sentiment_analysis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
